# Mars Scraping

Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.

In [1]:
import pandas as pd
import datetime as dt
from flask import Flask
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

### NASA Mars News

1. Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Courtney\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Retrieve the News Title
news_title = soup.body.find('div', class_='content_title')
news_title.text

'7 Things to Know About the Mars 2020 Perseverance Rover Mission'

In [6]:
# Retrieve the paragraph text
news_p = soup.find('div', class_='article_teaser_body')
news_p.text

"NASA's next rover to the Red Planet is slated to launch no earlier than July 30. These highlights will get you up to speed on the ambitious mission."

### JPL Mars Space Images - Featured Image

1. Visit the url for the Featured Space Image site https://spaceimages-mars.com/.
2. Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called 'featured_image_url'.
3. Make sure to find the image url to the full size .jpg image.
4. Make sure to save a complete url string for this image.

In [7]:
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [8]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [9]:
image_url = soup.find('img', class_='headerimage')['src']
featured_image_url = f'https://spaceimages-mars.com/{image_url}'
featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts

Visit the Mars Facts webpage https://galaxyfacts-mars.com/ and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.


Use Pandas to convert the data to a HTML table string.

In [10]:
#defining target url
url =  'https://galaxyfacts-mars.com/'

In [11]:
# use pandas to read in html from previously defined url
facts = pd.read_html(url)

# grabbing the second table with only Mars facts
df = facts[1]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
# #resetting dataframe index with fact attributes
# df = df.set_index(0)

# #renaming the index
# df.index.rename('Mars Attributes', inplace=True)

#renaming columns
df.columns=['Mars Attributes', 'Facts']
df

,Mars Attributes,Facts
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
#converting df to html table for later use
df.to_html('mars_facts_table.html')

### Mars Hemispheres

Visit the astrogeology site https://marshemispheres.com/ to obtain high resolution images for each of Mar's hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [19]:
url = 'https://marshemispheres.com/'
browser.visit(url)

In [20]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [21]:
hemisphere_image_urls = []
results = soup.find_all('div', class_='item')
for result in results:
    images = {}
    title = result.find('h3').text
    images['title'] = title[:-9]
    browser.find_by_text(title).click()
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    images['url'] = soup.find('img', class_='wide-image')['src']
    browser.back()
    hemisphere_image_urls.append(images)  

In [22]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'url': 'images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'url': 'images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'url': 'images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'url': 'images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [27]:
browser.quit()

## Step 2 - MongoDB and Flask Application

1. Start by converting your Jupyter notebook into a Python script called scrape_mars.py with a function called scrape that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.